In [1]:
import numpy as np
import pandas as pd
import datetime
import os
import json
import sklearn
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, KFold
import math

In [2]:
raw_data = pd.read_csv('training.csv')

In [5]:
raw_data.head()

,movie_id,cast,crew,budget,genres,homepage,keywords,original_language,original_title,overview,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,rating
0,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...",237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,3
1,285,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...",300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",3
2,206647,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de...",245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,3
3,49026,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de...",250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,3
4,49529,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de...",260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...","[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",3


In [6]:
print(list(raw_data))
print(raw_data.dtypes)
print(raw_data.shape)

['movie_id', 'cast', 'crew', 'budget', 'genres', 'homepage', 'keywords', 'original_language', 'original_title', 'overview', 'production_companies', 'production_countries', 'release_date', 'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'rating']
movie_id                  int64
cast                     object
crew                     object
budget                    int64
genres                   object
homepage                 object
keywords                 object
original_language        object
original_title           object
overview                 object
production_companies     object
production_countries     object
release_date             object
revenue                   int64
runtime                 float64
spoken_languages         object
status                   object
tagline                  object
rating                    int64
dtype: object
(2100, 19)


In [8]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2100 entries, 0 to 2099
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   movie_id              2100 non-null   int64  
 1   cast                  2100 non-null   object 
 2   crew                  2100 non-null   object 
 3   budget                2100 non-null   int64  
 4   genres                2100 non-null   object 
 5   homepage              955 non-null    object 
 6   keywords              2100 non-null   object 
 7   original_language     2100 non-null   object 
 8   original_title        2100 non-null   object 
 9   overview              2100 non-null   object 
 10  production_companies  2100 non-null   object 
 11  production_countries  2100 non-null   object 
 12  release_date          2100 non-null   object 
 13  revenue               2100 non-null   int64  
 14  runtime               2100 non-null   float64
 15  spoken_languages     

In [169]:
# Usage and explaination of each feature

# Movie ID -> delete
# Cast -> each character name -> too many -> delete -> possible select top 10 characters
# Crew -> each worker name -> too many -> delete -> possible select top 10 workers
# budget -> numeric number -> no process -> use
# genres -> 18 uniq values in total -> put into one hot encoding
# homepage -> null values in -> hasHomepage = 0/1
# keywords -> too many -> delete -> possible select top 10 characters
# original_language -> possibly noise -> because EN dominate the training set
# original_title -> word tokens 
# overview -> word tokens
# production_companies -> select top 10 -> one hot encoding
# production_countries -> top 15
# release_date -> into year, weekdays, month, weekofyear, day, quarter
# runtime -> direct use
# spoken_languages -> top 10
# status -> all same value -> useless
# tagline -> words -> too many


raw_data['spoken_languages'][0]

'[{"iso_639_1": "en", "name": "English"}, {"iso_639_1": "es", "name": "Espa\\u00f1ol"}]'

In [170]:
raw_data['spoken_languages'][1]

'[{"iso_639_1": "en", "name": "English"}]'

In [31]:
# # helper functions
def get_genres(x):
    json_data = json.loads(x)
    list_genres = []
    for token in json_data:
        list_genres.append(token['name'])
    return list_genres

In [34]:
df_dataset = pd.DataFrame(columns=['movie_id'])
df_dataset['movie_id'] = raw_data['movie_id']
df_dataset['budget'] = raw_data['budget']
df_dataset['genres'] = raw_data['genres'].apply(lambda row: get_genres(row))

In [35]:
df_dataset

,movie_id,budget,genres
0,19995,237000000,"[Action, Adventure, Fantasy, Science Fiction]"
1,285,300000000,"[Adventure, Fantasy, Action]"
2,206647,245000000,"[Action, Adventure, Crime]"
3,49026,250000000,"[Action, Crime, Drama, Thriller]"
4,49529,260000000,"[Action, Adventure, Science Fiction]"
...,...,...,...
2095,9594,15000000,"[Thriller, Action, Crime, Drama]"
2096,4638,12000000,"[Crime, Action, Comedy]"
2097,13972,16000000,"[Comedy, Drama, Romance]"
2098,5038,15000000,"[Drama, Romance]"


In [36]:
unique_genres = df_dataset["genres"].apply(pd.Series).stack().unique()
print("Number of genres: {}".format(len(unique_genres)))
print("Genres: {}".format(unique_genres))

Number of genres: 18
Genres: ['Action' 'Adventure' 'Fantasy' 'Science Fiction' 'Crime' 'Drama'
 'Thriller' 'Animation' 'Family' 'Western' 'Comedy' 'Romance' 'Horror'
 'Mystery' 'History' 'War' 'Music' 'Documentary']


In [40]:
genres_dummies = pd.get_dummies(df_dataset["genres"].apply(pd.Series).stack()).sum(level=0) #one hot encoding
genres_dummies.head()

,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,History,Horror,Music,Mystery,Romance,Science Fiction,Thriller,War,Western
0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0
1,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0
4,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [41]:
df_dataset = pd.concat([df_dataset, genres_dummies],axis=1, sort=False) #merging two data frame
df_dataset.head(5)

,movie_id,budget,genres,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,...,Fantasy,History,Horror,Music,Mystery,Romance,Science Fiction,Thriller,War,Western
0,19995,237000000,"[Action, Adventure, Fantasy, Science Fiction]",1,1,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0
1,285,300000000,"[Adventure, Fantasy, Action]",1,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,206647,245000000,"[Action, Adventure, Crime]",1,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,49026,250000000,"[Action, Crime, Drama, Thriller]",1,0,0,0,1,0,1,...,0,0,0,0,0,0,0,1,0,0
4,49529,260000000,"[Action, Adventure, Science Fiction]",1,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [42]:
# add homepage
df_dataset['has_homepage'] = 0
df_dataset.loc[raw_data['homepage'].isnull() == False, 'has_homepage'] = 1 #1 here means it has home page

In [43]:
df_dataset

,movie_id,budget,genres,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,...,History,Horror,Music,Mystery,Romance,Science Fiction,Thriller,War,Western,has_homepage
0,19995,237000000,"[Action, Adventure, Fantasy, Science Fiction]",1,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
1,285,300000000,"[Adventure, Fantasy, Action]",1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,206647,245000000,"[Action, Adventure, Crime]",1,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
3,49026,250000000,"[Action, Crime, Drama, Thriller]",1,0,0,0,1,0,1,...,0,0,0,0,0,0,1,0,0,1
4,49529,260000000,"[Action, Adventure, Science Fiction]",1,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2095,9594,15000000,"[Thriller, Action, Crime, Drama]",1,0,0,0,1,0,1,...,0,0,0,0,0,0,1,0,0,0
2096,4638,12000000,"[Crime, Action, Comedy]",1,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,1
2097,13972,16000000,"[Comedy, Drama, Romance]",0,0,0,1,0,0,1,...,0,0,0,0,1,0,0,0,0,1
2098,5038,15000000,"[Drama, Romance]",0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0


In [53]:
# Original language one hot encoding

unique_language = raw_data["original_language"].apply(pd.Series).stack().unique()
print("Number of genres: {}".format(len(unique_language)))
print("Genres: {}".format(unique_language))
genres_dummies = pd.get_dummies(raw_data["original_language"].apply(pd.Series).stack()).sum(level=0) #one hot encoding
genres_dummies.head()
df_dataset = pd.concat([df_dataset, genres_dummies],axis=1, sort=False) #merging two data frame
df_dataset.head(5)

Number of genres: 12
Genres: ['en' 'ja' 'fr' 'zh' 'ko' 'te' 'ru' 'de' 'cn' 'es' 'it' 'nl']


,movie_id,budget,genres,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,...,en,es,fr,it,ja,ko,nl,ru,te,zh
0,19995,237000000,"[Action, Adventure, Fantasy, Science Fiction]",1,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,285,300000000,"[Adventure, Fantasy, Action]",1,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,206647,245000000,"[Action, Adventure, Crime]",1,1,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
3,49026,250000000,"[Action, Crime, Drama, Thriller]",1,0,0,0,1,0,1,...,1,0,0,0,0,0,0,0,0,0
4,49529,260000000,"[Action, Adventure, Science Fiction]",1,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [56]:
df_dataset['production_companies'] = raw_data['production_companies'].apply(lambda row: get_genres(row))

In [57]:
df_dataset['production_companies']

0       [Ingenious Film Partners, Twentieth Century Fo...
1       [Walt Disney Pictures, Jerry Bruckheimer Films...
2                        [Columbia Pictures, Danjaq, B24]
3       [Legendary Pictures, Warner Bros., DC Entertai...
4                                  [Walt Disney Pictures]
                              ...                        
2095                               [Stone Group Pictures]
2096    [Universal Pictures, Big Talk Productions, Stu...
2097                                    [New Line Cinema]
2098    [The Weinstein Company, Gravier Productions, A...
2099                               [Punch 21 Productions]
Name: production_companies, Length: 2100, dtype: object

In [58]:
unique_companies = df_dataset['production_companies'].apply(pd.Series).stack().unique()
print("Number of companies: {}".format(len(unique_companies)))
print("Companies: {}".format(unique_companies))

Number of companies: 2347
Companies: ['Ingenious Film Partners' 'Twentieth Century Fox Film Corporation'
 'Dune Entertainment' ... 'Stone Group Pictures' 'Antena 3 Televisión'
 'Punch 21 Productions']


In [60]:
df_dataset['production_companies'].apply(pd.Series).stack().value_counts().head(10)

Warner Bros.                              242
Universal Pictures                        218
Paramount Pictures                        184
Twentieth Century Fox Film Corporation    155
Columbia Pictures                         148
New Line Cinema                           104
Relativity Media                           88
Walt Disney Pictures                       87
Touchstone Pictures                        79
Village Roadshow Pictures                  70
dtype: int64

In [80]:
genres_dummies = pd.get_dummies(df_dataset['production_companies'].apply(pd.Series).stack()).sum(level=0)

In [82]:
genres_dummies = genres_dummies[['Warner Bros.', 'Universal Pictures', 'Paramount Pictures','Twentieth Century Fox Film Corporation', 'Columbia Pictures', 'New Line Cinema', 'Relativity Media', 'Walt Disney Pictures', 'Touchstone Pictures', 'Village Roadshow Pictures']]

In [83]:
genres_dummies

,Warner Bros.,Universal Pictures,Paramount Pictures,Twentieth Century Fox Film Corporation,Columbia Pictures,New Line Cinema,Relativity Media,Walt Disney Pictures,Touchstone Pictures,Village Roadshow Pictures
0,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,1,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...
2095,0,0,0,0,0,0,0,0,0,0
2096,0,1,0,0,0,0,0,0,0,0
2097,0,0,0,0,0,1,0,0,0,0
2098,0,0,0,0,0,0,0,0,0,0


In [84]:
df_dataset.index.difference(genres_dummies.index)

Int64Index([1565, 1899], dtype='int64')

In [86]:
add_df = pd.DataFrame(index=[1565, 1899], columns=genres_dummies.columns).fillna(0)

In [87]:
add_df

,Warner Bros.,Universal Pictures,Paramount Pictures,Twentieth Century Fox Film Corporation,Columbia Pictures,New Line Cinema,Relativity Media,Walt Disney Pictures,Touchstone Pictures,Village Roadshow Pictures
1565,0,0,0,0,0,0,0,0,0,0
1899,0,0,0,0,0,0,0,0,0,0


In [93]:
genres_dummies = pd.concat([genres_dummies, add_df])

In [102]:
genres_dummies

,Warner Bros.,Universal Pictures,Paramount Pictures,Twentieth Century Fox Film Corporation,Columbia Pictures,New Line Cinema,Relativity Media,Walt Disney Pictures,Touchstone Pictures,Village Roadshow Pictures
0,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,1,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...
2097,0,0,0,0,0,1,0,0,0,0
2098,0,0,0,0,0,0,0,0,0,0
2099,0,0,0,0,0,0,0,0,0,0
1565,0,0,0,0,0,0,0,0,0,0


In [103]:
df = genres_dummies.reset_index()

In [107]:
df.sort_values(['index'],inplace=True)

In [114]:
df.set_index(['index'], inplace=True)

In [115]:
df

,Warner Bros.,Universal Pictures,Paramount Pictures,Twentieth Century Fox Film Corporation,Columbia Pictures,New Line Cinema,Relativity Media,Walt Disney Pictures,Touchstone Pictures,Village Roadshow Pictures
index,,,,,,,,,,
0,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,1,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...
2095,0,0,0,0,0,0,0,0,0,0
2096,0,1,0,0,0,0,0,0,0,0
2097,0,0,0,0,0,1,0,0,0,0


In [110]:
df_dataset = pd.concat([df_dataset, df],axis=1, sort=False)

In [119]:
df_dataset.drop(inplace= True, axis=1, columns=['index','Warner Bros.', 'Universal Pictures', 'Paramount Pictures','Twentieth Century Fox Film Corporation', 'Columbia Pictures', 'New Line Cinema', 'Relativity Media', 'Walt Disney Pictures', 'Touchstone Pictures', 'Village Roadshow Pictures'])

In [121]:
df_dataset = pd.concat([df_dataset, df],axis=1, sort=False)

In [126]:
df_dataset.iloc[2097]

movie_id                                                     13972
budget                                                    16000000
genres                                    [Comedy, Drama, Romance]
Action                                                           0
Adventure                                                        0
Animation                                                        0
Comedy                                                           1
Crime                                                            0
Documentary                                                      0
Drama                                                            1
Family                                                           0
Fantasy                                                          0
History                                                          0
Horror                                                           0
Music                                                         

In [129]:
# create countries
df_dataset['production_countries'] = raw_data['production_countries'].apply(lambda row: get_genres(row))
df_dataset['production_countries']

0       [United States of America, United Kingdom]
1                       [United States of America]
2       [United Kingdom, United States of America]
3                       [United States of America]
4                       [United States of America]
                           ...                    
2095                    [United States of America]
2096                              [United Kingdom]
2097                    [United States of America]
2098             [United States of America, Spain]
2099                    [United States of America]
Name: production_countries, Length: 2100, dtype: object

In [130]:
unique_countries = df_dataset['production_countries'].apply(pd.Series).stack().unique()
print("Number of countries: {}".format(len(unique_countries)))
print("countries: {}".format(unique_countries))

Number of countries: 52
countries: ['United States of America' 'United Kingdom' 'Jamaica' 'Bahamas'
 'Dominica' 'Czech Republic' 'Poland' 'Slovenia' 'New Zealand' 'Germany'
 'China' 'Canada' 'Italy' 'Japan' 'Malta' 'Australia' 'France'
 'Netherlands' 'Spain' 'United Arab Emirates' 'India' 'Hong Kong' 'Taiwan'
 'Belgium' 'Ireland' 'Morocco' 'Hungary' 'Singapore' 'Norway' 'Sweden'
 'South Africa' 'Russia' 'Romania' 'Mexico' 'Pakistan' 'Finland' 'Iceland'
 'Denmark' 'Tunisia' 'Philippines' 'Bulgaria' 'Switzerland' 'South Korea'
 'Brazil' 'Peru' 'Luxembourg' 'Portugal' 'Serbia' 'Chile' 'Argentina'
 'Kazakhstan' 'Greece']


In [134]:
df_dataset['production_countries'].apply(pd.Series).stack().value_counts().head(15)

United States of America    1984
United Kingdom               285
Germany                      189
France                       120
Canada                        97
Australia                     61
China                         30
Italy                         27
Spain                         26
Japan                         25
Hong Kong                     18
New Zealand                   18
Czech Republic                16
India                         14
Ireland                       13
dtype: int64

In [135]:
genres_dummies = pd.get_dummies(df_dataset['production_countries'].apply(pd.Series).stack()).sum(level=0)

In [136]:
genres_dummies

,Argentina,Australia,Bahamas,Belgium,Brazil,Bulgaria,Canada,Chile,China,Czech Republic,...,South Africa,South Korea,Spain,Sweden,Switzerland,Taiwan,Tunisia,United Arab Emirates,United Kingdom,United States of America
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2095,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2096,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2097,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2098,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1


In [137]:
df_dataset.index.difference(genres_dummies.index)

Int64Index([269, 1710, 1745], dtype='int64')

In [138]:
genres_dummies = genres_dummies[['United States of America', 'United Kingdom', 'Germany', 'France', 'Canada', 'Australia', 'China', 'Italy', 'Spain', 'Japan', 'Hong Kong', 'New Zealand', 'Czech Republic', 'India', 'Ireland']]

In [139]:
genres_dummies

,United States of America,United Kingdom,Germany,France,Canada,Australia,China,Italy,Spain,Japan,Hong Kong,New Zealand,Czech Republic,India,Ireland
0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2095,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2096,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2097,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2098,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0


In [140]:
add_df = pd.DataFrame(index=[269, 1710, 1745], columns=genres_dummies.columns).fillna(0)

In [142]:
genres_dummies = pd.concat([genres_dummies, add_df])

In [143]:
genres_dummies

,United States of America,United Kingdom,Germany,France,Canada,Australia,China,Italy,Spain,Japan,Hong Kong,New Zealand,Czech Republic,India,Ireland
0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2098,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2099,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
269,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1710,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [144]:
df = genres_dummies.reset_index()
df.sort_values(['index'],inplace=True)
df.set_index(['index'], inplace=True)

In [147]:
df_dataset = pd.concat([df_dataset, df],axis=1, sort=False)

In [158]:
# release date process
df_dataset['release_date'] = raw_data['release_date']

In [160]:
df_dataset['release_date'] = pd.to_datetime(df_dataset['release_date'])

In [162]:
def process_date(df_date):
    '''this function add column like 
     year, weeekday, month and so on column
    and add prefix of release_date before 
    all the above column eg realease_date_year'''
    date_parts = ["year", "weekday", "month", 'weekofyear', 'day', 'quarter']
    for part in date_parts:
        part_col = 'release_date' + "_" + part #add prefix as  "release_date" before the columne
        df_dataset[part_col] = getattr(df_date['release_date'].dt, part).astype(int)
    
    return df_date

In [165]:
date_parts = ["year", "weekday", "month", 'weekofyear', 'day', 'quarter']
for part in date_parts:
    part_col = 'release_date' + "_" + part #add prefix as  "release_date" before the columne
    df_dataset[part_col] = getattr(df_dataset['release_date'].dt, part).astype(int)

<ipython-input-165-c5f11f4208c5>:4: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  df_dataset[part_col] = getattr(df_dataset['release_date'].dt, part).astype(int)


In [168]:
df_dataset['runtime'] = raw_data['runtime']

In [184]:
# get spoken language 

# # helper functions
def get_spoken_language(x):
    json_data = json.loads(x)
    list_genres = []
    for token in json_data:
        list_genres.append(str(str("spoken_")+token['iso_639_1']))
    return list_genres

In [191]:
df_dataset['spoken_languages'] = raw_data['spoken_languages'].apply(lambda row: get_spoken_language(row))

In [192]:
df_dataset['spoken_languages']

0                                  [spoken_en, spoken_es]
1                                             [spoken_en]
2       [spoken_fr, spoken_en, spoken_es, spoken_it, s...
3                                             [spoken_en]
4                                             [spoken_en]
                              ...                        
2095                                          [spoken_en]
2096                                          [spoken_en]
2097                                          [spoken_en]
2098                               [spoken_es, spoken_en]
2099                                          [spoken_en]
Name: spoken_languages, Length: 2100, dtype: object

In [193]:
unique_spoken_language = df_dataset['spoken_languages'].apply(pd.Series).stack().unique()
print("Number of spoken_language: {}".format(len(unique_spoken_language)))
print("spoken_language: {}".format(unique_spoken_language))

# genres_dummies = pd.get_dummies(raw_data["original_language"].apply(pd.Series).stack()).sum(level=0) #one hot encoding
# genres_dummies.head()
# df_dataset = pd.concat([df_dataset, genres_dummies],axis=1, sort=False) #merging two data frame
# df_dataset.head(5)

Number of spoken_language: 66
spoken_language: ['spoken_en' 'spoken_es' 'spoken_fr' 'spoken_it' 'spoken_de' 'spoken_tr'
 'spoken_el' 'spoken_zh' 'spoken_th' 'spoken_is' 'spoken_ru' 'spoken_sv'
 'spoken_ro' 'spoken_ja' 'spoken_la' 'spoken_hi' 'spoken_pt' 'spoken_bo'
 'spoken_fa' 'spoken_ur' 'spoken_ar' 'spoken_sa' 'spoken_gd' 'spoken_cs'
 'spoken_cn' 'spoken_ko' 'spoken_no' 'spoken_ta' 'spoken_nv' 'spoken_he'
 'spoken_da' 'spoken_nl' 'spoken_af' 'spoken_ga' 'spoken_so' 'spoken_fi'
 'spoken_bg' 'spoken_yi' 'spoken_vi' 'spoken_hu' 'spoken_uk' 'spoken_eo'
 'spoken_pl' 'spoken_et' 'spoken_sq' 'spoken_hr' 'spoken_tl' 'spoken_sh'
 'spoken_sk' 'spoken_ml' 'spoken_te' 'spoken_cy' 'spoken_hy' 'spoken_xh'
 'spoken_km' 'spoken_sw' 'spoken_ny' 'spoken_st' 'spoken_zu' 'spoken_kw'
 'spoken_sr' 'spoken_si' 'spoken_ne' 'spoken_ps' 'spoken_kk' 'spoken_mn']


In [196]:
df_dataset['spoken_languages'].apply(pd.Series).stack().value_counts().head(10)

spoken_en    2071
spoken_fr     193
spoken_es     180
spoken_de     122
spoken_ru     113
spoken_it      96
spoken_zh      63
spoken_ja      58
spoken_ar      42
spoken_pt      34
dtype: int64

In [199]:
genres_dummies = pd.get_dummies(df_dataset['spoken_languages'].apply(pd.Series).stack()).sum(level=0)

In [200]:
genres_dummies

,spoken_af,spoken_ar,spoken_bg,spoken_bo,spoken_cn,spoken_cs,spoken_cy,spoken_da,spoken_de,spoken_el,...,spoken_th,spoken_tl,spoken_tr,spoken_uk,spoken_ur,spoken_vi,spoken_xh,spoken_yi,spoken_zh,spoken_zu
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2095,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2096,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2097,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2098,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [201]:
df_dataset.index.difference(genres_dummies.index)

Int64Index([1086], dtype='int64')

In [205]:
add_df = pd.DataFrame(index=[1086], columns=genres_dummies.columns).fillna(0)
genres_dummies = pd.concat([genres_dummies, add_df])

In [206]:
genres_dummies

,spoken_af,spoken_ar,spoken_bg,spoken_bo,spoken_cn,spoken_cs,spoken_cy,spoken_da,spoken_de,spoken_el,...,spoken_th,spoken_tl,spoken_tr,spoken_uk,spoken_ur,spoken_vi,spoken_xh,spoken_yi,spoken_zh,spoken_zu
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2096,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2097,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2098,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2099,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [207]:
genres_dummies = genres_dummies[['spoken_en', 'spoken_fr', 'spoken_es', 'spoken_de', 'spoken_ru', 'spoken_it', 'spoken_zh', 'spoken_ja', 'spoken_ar', 'spoken_pt']]

In [209]:
df = genres_dummies.reset_index()
df.sort_values(['index'],inplace=True)
df.set_index(['index'], inplace=True)

In [210]:
df_dataset = pd.concat([df_dataset, df],axis=1, sort=False)

In [215]:
df_dataset.iloc[1086][['spoken_en', 'spoken_fr', 'spoken_es', 'spoken_de', 'spoken_ru', 'spoken_it', 'spoken_zh', 'spoken_ja', 'spoken_ar', 'spoken_pt']]

spoken_en    0
spoken_fr    0
spoken_es    0
spoken_de    0
spoken_ru    0
spoken_it    0
spoken_zh    0
spoken_ja    0
spoken_ar    0
spoken_pt    0
Name: 1086, dtype: object

In [218]:
df_dataset.shape

(2100, 80)

In [220]:
df_dataset.to_csv("training_set_processed.csv", index=False)